In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/project

/content/drive/MyDrive/project


In [ ]:
from string import ascii_uppercase
#from draw_utils import *
#from pyglet.gl import *
import numpy as np
import pandas as pd
import os
import copy


# reward
move_reward = -0.1
obs_reward = -0.2
goal_reward = 10
finish_reward = 20
print('reward:' , move_reward, obs_reward, goal_reward)

local_path = os.path.abspath(os.path.join(os.path.dirname('__file__')))


class Simulator:
    def __init__(self):
        '''
        height : 그리드 높이
        width : 그리드 너비 
        inds : A ~ Q alphabet list
        '''
        # Load train data
        self.files = pd.read_csv(os.path.join(local_path, "./data/train1.csv")) #"./data/factory_order_train.csv"))
        self.height = 10
        self.width = 9
        self.inds = list(ascii_uppercase)[:17]

    def set_box(self):
        '''
        아이템들이 있을 위치를 미리 정해놓고 그 위치 좌표들에 아이템이 들어올 수 있으므로 그리드에 100으로 표시한다.
        데이터 파일에서 이번 에피소드 아이템 정보를 받아 가져와야 할 아이템이 있는 좌표만 -100으로 표시한다.
        self.local_target에 에이전트가 이번에 방문해야할 좌표들을 저장한다.
        따라서 가져와야하는 아이템 좌표와 end point 좌표(처음 시작했던 좌표로 돌아와야하므로)가 들어가게 된다.
        '''
        box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))

        # 물건이 들어있을 수 있는 경우
        for box in box_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(box, "row")][getattr(box, "col")] = 0

        # 물건이 실제 들어있는 경우
        order_item = list(set(self.inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]
        
        for order_box in order_csv.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = 200
            # local target에 가야 할 위치 좌표 넣기
            self.local_target.append(
                [getattr(order_box, "row"),
                 getattr(order_box, "col")]
                )

        #self.local_target.append([9,4]) 
        # 알파벳을 Grid에 넣어서 -> grid에 2Dconv 적용 가능

    def set_obstacle(self):
        '''
        장애물이 있어야하는 위치는 미리 obstacles.csv에 정의되어 있다. 이 좌표들을 0으로 표시한다.
        '''
        obstacles_data = pd.read_csv(os.path.join(local_path, "./data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = 0

    def reset(self, epi):
        '''
        reset()은 첫 스텝에서 사용되며 그리드에서 에이전트 위치가 start point에 있게 한다.

        :param epi: episode, 에피소드 마다 가져와야 할 아이템 리스트를 불러올 때 사용
        :return: 초기셋팅 된 그리드
        :rtype: numpy.ndarray
        _____________________________________________________________________________________
        items : 이번 에피소드에서 가져와야하는 아이템들
        terminal_location : 현재 에이전트가 찾아가야하는 목적지
        local_target : 한 에피소드에서 찾아가야하는 아이템 좌표, 마지막 엔드 포인트 등의 위치좌표들
        actions: visualization을 위해 에이전트 action을 저장하는 리스트
        curloc : 현재 위치
        '''

        # initial episode parameter setting
        self.epi = epi
        self.items = list(self.files.iloc[self.epi])[0]
        self.cumulative_reward = 0
        self.terminal_location = None
        self.local_target = []
        self.actions = []
        self.item_loc = False ## 수정
        
        # initial grid setting
        self.grid = np.ones((self.height, self.width), dtype="float16")

        # set information about the gridworld
        self.set_box()
        self.set_obstacle()

        # start point를 grid에 표시
        self.curloc = [9, 4]
        self.grid[int(self.curloc[0])][int(self.curloc[1])] = 100
        
        self.done = False
        
        return self.grid

    def apply_action(self, action, cur_x, cur_y):
        '''
        에이전트가 행한 action대로 현 에이전트의 위치좌표를 바꾼다.
        action은 discrete하며 4가지 up,down,left,right으로 정의된다.
        
        :param x: 에이전트의 현재 x 좌표
        :param y: 에이전트의 현재 y 좌표
        :return: action에 따라 변한 에이전트의 x 좌표, y 좌표
        :rtype: int, int
        '''
        new_x = cur_x
        new_y = cur_y
        # up
        if action == 0:
            new_x = cur_x - 1
        # down
        elif action == 1:
            new_x = cur_x + 1
        # left
        elif action == 2:
            new_y = cur_y - 1
        # right
        else:
            new_y = cur_y + 1

        return int(new_x), int(new_y)


    def get_reward(self, new_x, new_y, out_of_boundary):
        '''
        get_reward함수는 리워드를 계산하는 함수이며, 상황에 따라 에이전트가 action을 옳게 했는지 판단하는 지표가 된다.

        :param new_x: action에 따른 에이전트 새로운 위치좌표 x
        :param new_y: action에 따른 에이전트 새로운 위치좌표 y
        :param out_of_boundary: 에이전트 위치가 그리드 밖이 되지 않도록 제한
        :return: action에 따른 리워드
        :rtype: float
        '''

        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            reward = obs_reward
                       
        else:
            # 장애물에 부딪히는 경우 
            if self.grid[new_x][new_y] == 0:
                reward = obs_reward  

            # 현재 목표에 도달한 경우
            elif [new_x, new_y] in self.terminal_location:
                if [new_x, new_y] == [9, 4]:
                    reward = finish_reward
                else:
                    reward = goal_reward

            # 그냥 움직이는 경우 
            else:
                reward = move_reward

        return reward

    def step(self, action):
        ''' 
        에이전트의 action에 따라 step을 진행한다.
        action에 따라 에이전트 위치를 변환하고, action에 대해 리워드를 받고, 어느 상황에 에피소드가 종료되어야 하는지 등을 판단한다.
        에이전트가 endpoint에 도착하면 gif로 에피소드에서 에이전트의 행동이 저장된다.

        :param action: 에이전트 행동
        :return:
            grid, 그리드
            reward, 리워드
            cumulative_reward, 누적 리워드
            done, 종료 여부
            goal_ob_reward, goal까지 아이템을 모두 가지고 돌아오는 finish율 계산을 위한 파라미터

        :rtype: numpy.ndarray, float, float, bool, bool/str

        (Hint : 시작 위치 (9,4)에서 up말고 다른 action은 전부 장애물이므로 action을 고정하는 것이 좋음)
        '''

        self.terminal_location = copy.deepcopy(self.local_target)
        cur_x,cur_y = self.curloc
        self.actions.append((cur_x, cur_y))

        goal_ob_reward = False
        
        new_x, new_y = self.apply_action(action, cur_x, cur_y)

        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]

        # 바깥으로 나가는 경우 종료
        if any(out_of_boundary):
            pass
            #self.done = True
            #goal_ob_reward = True
        else:
            # 장애물에 부딪히는 경우 종료
            if self.grid[new_x][new_y] == 0:
                pass
                #self.done = True
                #goal_ob_reward = True

            # 현재 목표에 도달한 경우
            elif [new_x, new_y] in self.terminal_location:

                # end point 일 때
                if [new_x, new_y] == [9,4]:
                    
                    self.done = True
                    self.local_target.remove([new_x, new_y])
                
                # item 일때
                else:
                    self.local_target.remove([new_x, new_y])
                    if not self.local_target:
                        self.local_target.append([9,4])
                        self.grid[9][4] = 200
                
                if self.item_loc: #저번에가 item 이었던 자리었으면
                    self.grid[cur_x][cur_y] = 0
                    self.grid[new_x][new_y] = 100
                else:
                    self.grid[cur_x][cur_y] = 1
                    self.grid[new_x][new_y] = 100

                goal_ob_reward = True
                self.item_loc=True
                
                self.curloc = [new_x, new_y]
            else:
                # 그냥 움직이는 경우
                if self.item_loc:
                    self.grid[cur_x][cur_y] = 0
                    self.grid[new_x][new_y] = 100
                    self.item_loc = False

                else:
                    self.grid[cur_x][cur_y] = 1
                    self.grid[new_x][new_y] = 100
                    
                self.curloc = [new_x,new_y]
                
        reward = self.get_reward(new_x, new_y, out_of_boundary)
        self.cumulative_reward += reward

        # if self.done == True:
        #     if [new_x, new_y] == [9, 4]:
        #         if self.terminal_location[0] == [9, 4]:
        #                 # 완료되면 GIFS 저장

        #             if len(self.actions) < 500:
        #                 print(f'500번 안에 들어왔다! : {len(self.actions)}')
        #                 goal_ob_reward = 'finish'
        #                 height = 10
        #                 width = 9 
        #                 display = Display(visible=False, size=(width, height))
        #                 display.start()

        #                 start_point = (9, 4)
        #                 unit = 50
        #                 screen_height = height * unit
        #                 screen_width = width * unit
        #                 log_path = "./logsdqn"
        #                 data_path = "./data"
        #                 render_cls = Render(screen_width, screen_height, unit, start_point, data_path, log_path)
        #                 for idx, new_pos in enumerate(self.actions):
        #                     render_cls.update_movement(new_pos, idx+1)

        #                 render_cls.save_gif(self.epi)
        #                 render_cls.viewer.close()
        #                 display.stop()
        #             else:
        #                 print(len(self.actions))
        
        
        
        
        return self.grid, reward, self.cumulative_reward, self.done, goal_ob_reward

reward: -0.1 -0.2 10


In [ ]:
import collections
import pdb
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#from Sim import *



In [ ]:

## Replay buffer

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)     #sample 메서드 : buffer 중에 n 개만 뽑음
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
                                                        # state 값, action 값, reward 값, 다음 state 값, done_mask 값
                                                        # done_mask : 종료 상태의 Value 값을 마스킹해줍니다

        for transition in mini_batch: #우리가 뽑은 n 개의 미니배치들 하나씩에서
            s, a, r, s_prime, done_mask = transition #

            # 이로 보아 하나의 sample 에는 s,a,s_prime,done_mask 값이 담김을 확인할 수 있다
            s_lst.append(s)
            a_lst.append([a]) # 자료형이 list 임을 확인 가능, 여러개의 action 이 담겨있을 것이라 추측
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch. float), torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
        # s_lst, s_prime_lst 는 타입을 바꿔주었다. 데이터가 int 였기 때문에
        # 나머지는 tensor화

    def size(self):
        return len(self.buffer)




In [ ]:
class QnetCNN(nn.Module):
    def __init__(self):
        super(QnetCNN, self).__init__()
        
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1),
            nn.ReLU())
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1),
            nn.ReLU())
    
        #Flatten
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(32, 16)
        self.fc2 = nn.Linear(16, 4)
        
    def forward(self,x):
        x = torch.from_numpy(np.asarray(x)).float()
        x = torch.reshape(x, (-1, 1, 7, 7))
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)# 좌우상하 4개의 값을 반환
        return x

    def sample_action(self, obs, epsilon):
        out = self.forward(obs) 
        coin = random.random() 
        

        if coin < epsilon: 
            return random.randint(0,3)
        else:
            return out.argmax().item() 


In [ ]:
def train(q, q_target, memory, optimizer): 
    for i in range(10):
        s, a, r, s_prime, done_mask = memory.sample(batch_size) #32개를 버퍼에서 뽑아 모아 놓은 s,a,r,s_prime,done_mask
        q_out = q(s)                                            # s 값으로 다음 각 action 값들의 value 값 반환
        q_a = q_out.gather(1,a)                      #선택한 액션값들의 q(s,a) 반환
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)  # 다음 state의 각 q(s,a) 값 반환
        target = r + gamma * max_q_prime * done_mask # 배열 맞춰주기, 쓰러진 경우는 제거
a
        loss = F.smooth_l1_loss(q_a, target)                    # DQN 의 손실함수 계산 L1 유클리드

        optimizer.zero_grad()                                   # optimizer 의 모든 parameter 를 0으로 변환
        loss.backward()                                         # loss 에 대한 gradient 계산
        optimizer.step()                                        # 손실값을 바탕으로 Qnet 의 파라미터 업데이트


In [ ]:
def model_save(model_dict, epi):
    PATH = './weights/'
    torch.save({
            'model': model_dict,
            'epi': epi,
            }, PATH + 'all_0602_item1.tar')

In [ ]:
import time
from tqdm import tqdm

learning_rate = 0.0001
gamma = 0.99
buffer_limit  = 100000
batch_size = 128

USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

def main():

    env = Simulator()
    files = pd.read_csv("./data/train1.csv")#("./data/factory_order_train.csv")

    q = QnetCNN()
    q_target = QnetCNN()
    
    # PATH = './weights/'
    # checkpoint = torch.load(PATH+'all_0602_item1.tar')
    # q.load_state_dict(checkpoint['model'])
    # epi_num = checkpoint['epi']
    # q.eval()
    # 현재 Qnet 의 파라미터를 q_target 에 load
    q_target.load_state_dict(q.state_dict())      
    

    memory = ReplayBuffer()

    print_interval = 100
    score = 0.0
    
    optimizer = optim.Adam(q.parameters(), lr = learning_rate) 
    # loss 값을 바탕으로 업데이트할 비율 (q_target 말고 q 만 업데이트)
    
    # scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma)

    cnt = 0
    masking = False

    for epi in tqdm(range(300000)):
        #time.sleep(0.001)
        if epi%1000 == 0:
            model_save(q.state_dict(), epi)
            
#         if epi <= 39999:
        epsilon = max(0.01, 0.3 - 0.0005 * (epi//100))

            
        ep = epi%17
        
        
        
        s = env.reset(ep)
        obs = np.asarray(s, dtype=np.float32)
        w,h = np.where(obs==100)
        w,h = w[0], h[0]
        b = np.zeros((7,7))
        c = obs[max(0,w-3):w+4,max(0,h-3):h+4]
        n,m = np.where(c==100)
        n,m = n[0],m[0]
        if (n < 3) and (m < 3):
            b[max(0, 3-n):len(c)+3-n, max(0, 3-m):len(c[0])+3-m]= c
        elif (n<3) and (m>=3):
            b[max(0, 3-n):len(c)+3-n, max(0, 3-m):len(c[0])]= c
        elif (n>=3) and (m<3):
            b[max(0, 3-n):len(c), max(0, 3-m):len(c[0])+3-m]= c
        else:
            b[max(0, 3-n):len(c), max(0, 3-m):len(c[0])]= c

        a_step= 0
        done = False 
        first = True
        while not done:
            if first:
                a = 0
                first = False
                
            elif obs[9][4] == 50:
                a = 0
                
            else:
                a = q.sample_action(torch.from_numpy(b).float(), epsilon) 
            
            s_prime, r, cumul, done, goal_reward = env.step(a)
            array = s_prime
            s_prime = np.asarray(s_prime, dtype=np.float32)
            
            w,h = np.where(s_prime==100)
            w,h = w[0], h[0]
            b_prime = np.zeros((7,7))
            c = s_prime[max(0,w-3):w+4,max(0,h-3):h+4]
            n,m = np.where(c==100)
            n,m = n[0],m[0]
            if (n < 3) and (m < 3):
                b_prime[max(0, 3-n):len(c)+3-n, max(0, 3-m):len(c[0])+3-m]= c
            elif (n<3) and (m>=3):
                b_prime[max(0, 3-n):len(c)+3-n, max(0, 3-m):len(c[0])]= c
            elif (n>=3) and (m<3):
                b_prime[max(0, 3-n):len(c), max(0, 3-m):len(c[0])+3-m]= c
            else:
                b_prime[max(0, 3-n):len(c), max(0, 3-m):len(c[0])]= c
            
            if done:
                masking = True
            
            if masking : # 끝났으면 train 안함
                cnt +=1
                print(f'지금 다 먹고 {cnt}번째 도착완료 \n  그리고 총 보상은 : {cumul} ')
                masking = False
            
            
            if a_step == 200:
                done = True
                goal_reward= True
            
            
             
            done_mask = 0.0 if done else 1.0

            memory.put((b,a,r/100, b_prime, done_mask))
            obs = s_prime
            b = b_prime
            score += r
        
            if done : # 끝났으면 train 안함
                break

            a_step +=1 # a 의 시행횟수
 

        if memory.size() > 2000: 
            train(q, q_target, memory, optimizer) 

        if epi%print_interval==0 and epi != 0: 
            action = ['↑', '↓', '←', '→'][a]
            print(f'epi = {ep}\n 마지막 상태 :\n{obs}\n 이 때 한 행동 : {action}\n 마지막 보상 : {r}\n 총 받은 보상 :{cumul}\n end : {done}\n clear : {goal_reward}')
           # env.now_state()
            
            q_target.load_state_dict(q.state_dict()) #q_target 지금걸로 업데이트
            print(f"episode :{ep}, score = {score/print_interval}, n_buffer :{memory.size()} , eps : {epsilon*100}")
            score = 0.0

main()

True


  0%|          | 102/300000 [00:12<10:18:28,  8.08it/s]

epi = 15
 마지막 상태 :
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  0.   1.   1.   1.   1.   1.   1.   1.   0.]
 [200.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  0.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  0.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  1.   1.   0.   1.   0.   1.   0.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1. 100.   1.   1.   1.   1.   1.   1.]
 [  0.   0.   0.   0.   1.   0.   0.   0.   0.]]
 이 때 한 행동 : ↓
 마지막 보상 : -0.1
 총 받은 보상 :-34.09999999999999
 end : True
 clear : True
episode :15, score = -31.45599999999051, n_buffer :20301 , eps : 29.95


  0%|          | 153/300000 [00:18<10:27:44,  7.96it/s]

지금 다 먹고 1번째 도착완료 
  그리고 총 보상은 : 8.40000000000002 


...

epi = 11
 마지막 상태 :
[[200.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  0.   1. 100.   1.   1.   1.   1.   1.   0.]
 [  0.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  0.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  0.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  1.   1.   0.   1.   0.   1.   0.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  0.   0.   0.   0.   1.   0.   0.   0.   0.]]
 이 때 한 행동 : ←
 마지막 보상 : -0.1
 총 받은 보상 :-22.700000000000053
 end : True
 clear : True
episode :11, score = -24.756999999993358, n_buffer :100000 , eps : 21.349999999999998


  6%|▌         | 17402/300000 [37:27<10:17:15,  7.63it/s]

epi = 9
 마지막 상태 :
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  0.   1.   1.   1.   1.   1.   1.   1.   0.]
 [  0.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  0.   1.   0.   1.   0.   1.   0.   1. 200.]
 [  0.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  1.   1.   0.   1.   0. 100.   0.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  0.   0.   0.   0.   1.   0.   0.   0.   0.]]
 이 때 한 행동 : ←
 마지막 보상 : -0.2
 총 받은 보상 :-27.799999999999937
 end : True
 clear : True
episode :9, score = -25.58399999999303, n_buffer :100000 , eps : 21.299999999999997


  6%|▌         | 17440/300000 [37:32<10:08:09,  7.74it/s]


KeyboardInterrupt: ignored

NameError: ignored